In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.utils import class_weight

from sklearn.preprocessing import label_binarize

In [ ]:
# 1. Parameters
IMAGE_SIZE  = (299, 299)
BATCH_SIZE  = 32
train_dir = '../../data5/train'
test_dir  = '../../data5/test'

# Create data generators with VGG16-specific preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen  = ImageDataGenerator(preprocessing_function=preprocess_input))

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

# Using the test set as "validation_data" (not ideal practice)
# but shown here due to the 2-folder constraint:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

num_classes = train_generator.num_classes
class_labels = list(train_generator.class_indices.keys())

print("Number of classes:", num_classes)
print("Class labels:", class_labels)
# Compute Class Weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

In [ ]:
# include_top=False removes InceptionV3's default classifier
base_model = InceptionV3(
    weights='imagenet', 
    include_top=False, 
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
)

# Example: Freeze first 249 layers, unfreeze the rest
for layer in base_model.layers[:249]:
    layer.trainable = False
for layer in base_model.layers[249:]:
    layer.trainable = True

# -------------------------------------------------
# 4. Add a Custom Head (Penultimate Layer = Embeddings)
# -------------------------------------------------
x = base_model.output
x = GlobalAveragePooling2D()(x)   # shape: (batch_size, 2048)
x = Dense(256, activation='relu', name='penultimate_layer')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax', name='final_predictions')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, min_lr=1e-6)

history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=15,
    verbose=1,
    class_weight=class_weights,
    callbacks=[early_stopping, lr_scheduler]
)


In [ ]:
train_loss = history.history['loss']
train_acc  = history.history['accuracy']
val_loss   = history.history['val_loss']
val_acc    = history.history['val_accuracy']

epochs_range = range(len(train_loss))

plt.figure(figsize=(14, 5))

# Loss
plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_loss, label='Training Loss')
plt.plot(epochs_range, val_loss,   label='Validation Loss')
plt.title('Training vs Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs_range, train_acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc,   label='Validation Accuracy')
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


In [ ]:
val_loss, val_accuracy = model.evaluate(train_generator, verbose=0)
print(f"Train Accuracy (Keras model): {val_accuracy:.4f}")

test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)
print(f"Test Accuracy (Keras model): {test_accuracy:.4f}")

In [ ]:
# Predict class probabilities
y_pred_probs = model.predict(test_generator)
# Convert probabilities to class indices
y_pred = np.argmax(y_pred_probs, axis=1)

# True labels from the test generator
y_true = test_generator.classes

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_labels, yticklabels=class_labels)
plt.title("Confusion Matrix")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.show()


In [ ]:
# 9. Build Feature Extractor Model
#    (Outputs the 'penultimate_layer' embeddings)
# -------------------------------------------------
feature_extractor = Model(
    inputs=model.input,
    outputs=model.get_layer('penultimate_layer').output  # the 256-dim layer
)
feature_extractor.summary()


In [ ]:
# 10. Helper Function to Extract Features
# -------------------------------------------------
def extract_features_and_labels(generator, extractor):
    """
    Pass all images in 'generator' through 'extractor'
    to obtain feature vectors. Also return integer labels.
    """
    features_list = []
    labels_list   = []
    
    # Reset generator
    generator.reset()
    batches = int(np.ceil(generator.samples / generator.batch_size))
    
    for _ in range(batches):
        X_batch, y_batch = next(generator)
        batch_features = extractor.predict(X_batch)  # (batch_size, 256)
        features_list.append(batch_features)
        labels_list.append(y_batch)
    
    features = np.concatenate(features_list, axis=0)
    labels_onehot = np.concatenate(labels_list, axis=0)
    labels_int = np.argmax(labels_onehot, axis=1)
    return features, labels_int

In [ ]:
# 11. Extract Features for Train, Val, and Test
# -------------------------------------------------
X_train, y_train = extract_features_and_labels(train_generator, feature_extractor)
X_test,  y_test  = extract_features_and_labels(test_generator,  feature_extractor)

print("Train features shape:", X_train.shape)  # (num_train_samples, 256)
print("Test  features shape:", X_test.shape)

# -------------------------------------------------
# 12. Train AdaBoost on the Extracted Features
# -------------------------------------------------
# Example: Using DecisionTree with max_depth=1 as base estimator (common for AdaBoost)
ada_params = {
    'base_estimator': DecisionTreeClassifier(max_depth=1),
    'n_estimators': 100,
    'learning_rate': 0.5,
    'random_state': 42
}

ada_clf = AdaBoostClassifier(**ada_params)
ada_clf.fit(X_train, y_train)

# Evaluate on the test set (optional)
y_test_pred = ada_clf.predict(X_test)
val_acc_ada = accuracy_score(y_test, y_test_pred)
print("AdaBoost Validation Accuracy:", val_acc_ada)


In [ ]:
# 13. Final Evaluation on Test Set
# -------------------------------------------------
y_test_pred = ada_clf.predict(X_test)
test_acc_ada = accuracy_score(y_test, y_test_pred)
print("AdaBoost Test Accuracy:", test_acc_ada)

print("\nClassification Report (AdaBoost):")
print(classification_report(y_test, y_test_pred, target_names=class_labels))

cm = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix (AdaBoost):\n", cm)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_labels, yticklabels=class_labels)
plt.title("Confusion Matrix (AdaBoost)")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.show()

In [ ]:
# Save the trained model
model.save("../../models/hybrid_model_inception_adaboost.keras")